# Testing the implementation of the PDE signature kernel and its gradients
***

There exists multiple kernel tricks to approximate the signature kernel:
- kernel trick used in SigGPDE (*PDE signature kernel*) implemented in `gpsig.kernels_pde`
  * cuda implementation
  * cython implementation
- kernel trick used in GPSig (*truncated signature kernel*) implemented in `gpsig.kernels`

In this notebook:
- we validate the implementation of the *PDE signature kernel* for SigGPDE by comparing the results to those obtained with the *truncated signature kernel*
- we time the gradients computation

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import sys
sys.path.append('..') 

import numpy as np
import matplotlib.pyplot as plt
import gpsig
import tensorflow as tf

import time

Successfully loaded the Cuda PDE signature kernel operator



We start by generating some random data

In [2]:
num_examples = 100
len_examples = 20
num_features = 2
input_dim = len_examples*num_features
X = np.random.randn(num_examples, len_examples, num_features)
X = X.cumsum(axis=1)
X/=np.max(X)

***
### Validating the implementation of the *PDE signature kernel* (forward)


##### Computing the PDE signature kernel with the Cython operator

In [3]:
order = 3
kern_cpu = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='cython',order=order)
K_pde_cpu = kern_cpu.compute_Kdiag(X.reshape([num_examples, -1]))

##### Computing the PDE signature kernel with the Cuda operator

In [6]:
order = 3
kern_gpu = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='gpu_op',order=order)
K_pde_gpu = kern_gpu.compute_Kdiag(X.reshape([num_examples, -1]))

##### Computing the truncated signature kernel

We need to take a high number of levels to ensure we have a good approximation of the signature kernel

In [7]:
num_levels = 9
kern_trunc = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels, order=num_levels, normalization=False)
K_trunc = kern_trunc.compute_Kdiag(X.reshape([num_examples, -1]))

##### Comparing the results

In [8]:
K_diff = K_pde_cpu - K_trunc
print(np.allclose(K_pde_cpu, K_trunc,rtol=1e-4))

True


In [9]:
K_diff = K_pde_gpu - K_trunc
print(np.allclose(K_pde_cpu, K_trunc,rtol=1e-4))

True


***
### Computing the gradients of the *PDE signature kernel* (backward)


##### Computing with the Cython operator

In [12]:
with tf.Session() as sess:   
    X_ = tf.constant(X)
    res = gpsig.kernels_pde.Kdiag_python(X_)
    t = time.time()
    grad = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad = sess.run(grad)[0]
    print('time Cython: ',  np.round(time.time()-t,3), 's')

time Cython:  0.264 s


##### Computing with the Cuda operator

In [13]:
with tf.Session(graph=tf.Graph()) as sess: 
    init = tf.initialize_all_variables()
    sess.run(init)
    X_ = tf.constant(X)    
    kern = gpsig.kernels_pde.UntruncSignatureKernel(input_dim, num_features, implementation='gpu_op',order=order)
    
    t = time.time()
    res = kern.Kdiag(tf.reshape(X_,[-1,input_dim]))
    grad = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad = sess.run(grad)
    
    print('time Cuda: ',  np.round(time.time()-t,3), 's')

time Cuda:  0.339 s


##### Computing with the truncated kernel

In [15]:
with tf.Session(graph=tf.Graph()) as sess:  
    init = tf.initialize_all_variables()
    sess.run(init)
    kern_trunc = gpsig.kernels.SignatureLinear(input_dim, num_features, num_levels, order=num_levels, normalization=False)
    X_ = tf.constant(X)
    
    t = time.time()
    res = kern_trunc.Kdiag(tf.reshape(X_,[-1,input_dim]))
    grad_trunc = tf.gradients(res, X_,unconnected_gradients=tf.UnconnectedGradients.NONE)
    grad_trunc = sess.run(grad_trunc)[0]
    print('time Truncated: ', np.round(time.time()-t,3), 's')

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, i

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/lengthscales/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, i

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/sigma/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, in wrapp

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/sigma/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, in wrapp

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/variances/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, in w

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'SignatureLinear-b4d2f69e-20/variances/IsVariableInitialized:0' shape=() dtype=bool>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameterized.py", line 302, in _build
    self._prior_tensor = self._build_prior(priors)  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/core/node.py", line 156, in build
    self._build()  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/gpflow/params/parameter.py", line 373, in _build
    self._prior_tensor = prior  File "/anaconda/envs/test_sigGPDE/lib/python3.7/site-packages/tensorflow_core/python/util/tf_should_use.py", line 198, in w

time Truncated:  4.122 s
